# DYNAMIC RESOURCE ALLOCATION MODEL 

In [1]:
import cvxpy as cvx
import pandas as pd
import numpy as np
from math import ceil

### Data Initializing & Processing

In [10]:
#set the number of hours to solve for
numhours =112

#set the number of loading docks in the facility
numdoors = 129

#create ranges for hours and doors
hours = range(1,numhours+1)
doors = range(1,numdoors+1)

#set how many areas in the facility (test model had a max size of 41 doors)
numareas = 7

areas = [1,2,3,4,5,6,7]
#set goal load per associate for each area
goal = {
    1: 266,
    2: 266, 
    3: 170,
    4: 266,
    5: 170,
    6: 266,
    7: 170}

#set the last door in each area
lastDoorInArea = {
    1: 41,
    2: 56,
    3: 70,
    4: 102,
    5: 111,
    6: 116,
    7: 129}

#create the sets of doors that make up each area
areaDoors = {}
for area in areas:
    if area == 1:
        areaDoors[area] = range(1,lastDoorInArea[area]+1)
    else: 
        areaDoors[area] = range(lastDoorInArea[area-1]+1,lastDoorInArea[area]+1)

#set the limit of load size for any given associate
loadLimit = 450

#set the limit on the number of lines any given associate can be assigned
lineLimit = 17

#pull in volume matrix
volData = 'volDataFull.csv'
volData = pd.read_csv(volData,header=None)
Vol = pd.DataFrame(volData,index=doors,columns=hours)

#create subsets of volume by area of the facility, such that you have a volume for any given area,door,hour combination
#areaAreaDoorHour
Vol2 = {'':0} 
for a in areas:
    for i in areaDoors[a]:
        for h in hours:
            Vol2[a,i,h] = Vol.loc[i,h]
del Vol2['']

#initialize sets for solutions, assignments and numassociates
solutions = {}
assignments = {}
numassociates = {}

In [11]:
for hour in hours:
    for area in areas:
        #determine the number of associates required by volume for the hour
        numassociates[area,hour] = int(ceil(sum(Vol2[area,door,hour] for door in areaDoors[area] if Vol2[area,door,hour] > 0)/goal[area]))

        #set the range of associates
        associates = range(1,numassociates[area,hour]+1)

        ''' ----- DECISION VARIABLES -----  '''

        #Assignment of doors to associates
        #X = m.addVars(areaDoors[area],associates,vtype=GRB.BINARY,name='X')
        X = {}
        Z = {}
        alpha = {}

        for j in associates:
            Z[j] = cvx.Variable(boolean=True)
            alpha[j] = cvx.Variable()
            for i in areaDoors[area]:
                X[i,j] = cvx.Variable(boolean=True)

        ''' ----- OBJECTIVE FUNCTION ----- '''

        obj = cvx.Minimize(sum(alpha[j] for j in associates))

        ''' ----- CONSTRAINTS ----- '''

        '''For any associate in the set of associates that are calculated as needed in that area and hour, 
        the sum of volume going to the doors that associate is assigned must be greater than or equal to 
        the goal, minus the underachievement.'''
        devFromGoal = [sum(Vol2[area,i, hour] * X[i,j] for i in areaDoors[area] if Vol2[area, i, hour] > 0) 
            >=  Z[j]*goal[area] - alpha[j] for j in associates]

        '''Ensure that associates are only assigned to doors that are adjacent to each other. 
        If a door has a volume of zero, it will skip the consideration of assigning an associate 
        to that door, and thus the next adjacent door can be assigned.'''
        doorAdjacency = [X[i2,j] >= X[i1,j] + X[i3,j] - 1 
            for j in associates for i1 in areaDoors[area] if Vol2[area,i1,hour]>0 
            for i2 in areaDoors[area] if Vol2[area,i2,hour]>0 for i3 in areaDoors[area] if Vol2[area,i3,hour]>0 
            if i1<i2<i3]

        '''Ensure that a door is assigned to one associate and only one.'''
        assignOnly1 = [sum(X[i,j] for j in associates) == 1 for i in areaDoors[area] if Vol2[area, i, hour] > 0]

        '''Ensure that if a door has no volume in a given hour, it must not be assigned.'''
        doNotAssignZeroVolDoor = [sum(X[i,j] for j in associates) == 0 for i in areaDoors[area] if Vol2[area, i, hour] == 0]

        '''Binary switching constraints. If a door is assigned an associate, that associate must be considered as used.'''
        usedBinarySwitch = [X[i,j] <= Z[j] for j in associates for i in areaDoors[area] if Vol2[area, i, hour] > 0]

        '''If an associate is to be used, they must be assigned to at least one door.'''
        assignedBinarySwitch = [sum(X[i,j] for i in areaDoors[area]) >= Z[j] for j in associates]

        '''Ensure that the deviation for an associate does not exceed the goal for that associate (don't completely understand this) '''
        IDK = [alpha[j] <= goal[area]*Z[j] for j in associates]

        '''The model has to use the number of associates that are calculated as needed in the area and hour,
        minus one associate if it can find a way to feasibly do so.'''
        minusOneIfPossible = [sum(Z[j] for j in associates) >= numassociates[area,hour]-1]       

        '''Ensure that the volume assigned to an associate across doors to not exceed a certain number of cartons.'''
        loadLimitz = [sum(Vol2[area,i,hour] * X[i,j] for i in areaDoors[area]) <= loadLimit for j in associates]        

        '''Ensure that the number of doors an associate is assigned never exceeds a certain number.'''
        lineLimit = [sum(X[i,j] for i in areaDoors[area]) <= lineLimit for j in associates]

        #XLB = [X[i,j]>=0 for i in areaDoors[area] for j in associates]
        #XUB = [X[i,j]<=1 for i in areaDoors[area] for j in associates]

        ZLB = [Z[j]>=0 for j in associates]
        alphaLB = [alpha[j]>=0 for j in associates]

        constraints = devFromGoal+doorAdjacency+assignOnly1+doNotAssignZeroVolDoor+usedBinarySwitch+assignedBinarySwitch+IDK+minusOneIfPossible+loadLimitz+lineLimit+ZLB+alphaLB


        ''' ----- BEGIN RUN ----- '''
        print('Hour: ',hour)
        print('Area: ',area)

        #allow optimization to run in the background without displaying all the output; solve
        prob = cvx.Problem(obj,constraints)
        prob.solve(solver='ECOS_BB',mi_max_iters=1)

        ''' ----- POST-SOLUTION PROCESSING ----- '''

        #calculate the load per used associate and whether associate was used
        load = {}
        used = {}
        totalUsed = 0
        for j in associates: 
            load[area,hour,j] = sum(Vol2[area,i,hour]*X[i,j].value for i in areaDoors[area] if Vol2[area,i,hour]>0 if not pd.isnull(X[i,j].value))
            if load[area,hour,j] > 0.2:
                used[area,hour,j] = 1
            else: 
                used[area,hour,j] = 0
            totalUsed += used[area,hour,j]

        #create a dataFrame for the hour, which will contain the associate to door assignments
        assignments[area,hour] = pd.DataFrame(data=None,index=areaDoors[area],columns=associates)    
        for i in areaDoors[area]:
             for j in associates:
                if type(X[i,j].value) == float:
                    if X[i,j].value > 0.2:
                        assignments[area,hour].loc[i,j] = X[i,j].value
                    else:
                        assignments[area,hour].loc[i,j] = ''
                    '''for assignment in assignments:
                        fileName = "".join(["resultsArea",str(area),"Hour",str(hour),".csv"])
                        assignments[assignment].to_csv(fileName)'''
        #calculate the area productivity
        productivity = {}

        ''' ----- DISPLAY OUTPUT ----- '''

        print('# Associates Required by Volume: ',numassociates[area,hour])
        print('# Associates Used: ',totalUsed)
        print('Assignments:')
        print(assignments[area,hour])
        print('Load per Associate: ')
        for j in associates:
            #print j,alpha[j].value
            print(j,'(',used[area,hour,j],')',': ',load[area,hour,j],'|',int(alpha[j].value))
        print('Total volume not expected to load (objective): ',int(prob.value))
        print('++++++++++++++++++++')
        print('')

Hour:  1
Area:  1
# Associates Required by Volume:  7
# Associates Used:  0
Assignments:
      1    2    3    4    5    6    7
1   NaN  NaN  NaN  NaN  NaN  NaN  NaN
2   NaN  NaN  NaN  NaN  NaN  NaN  NaN
3   NaN  NaN  NaN  NaN  NaN  NaN  NaN
4   NaN  NaN  NaN  NaN  NaN  NaN  NaN
5   NaN  NaN  NaN  NaN  NaN  NaN  NaN
6   NaN  NaN  NaN  NaN  NaN  NaN  NaN
7   NaN  NaN  NaN  NaN  NaN  NaN  NaN
8   NaN  NaN  NaN  NaN  NaN  NaN  NaN
9   NaN  NaN  NaN  NaN  NaN  NaN  NaN
10  NaN  NaN  NaN  NaN  NaN  NaN  NaN
11  NaN  NaN  NaN  NaN  NaN  NaN  NaN
12  NaN  NaN  NaN  NaN  NaN  NaN  NaN
13  NaN  NaN  NaN  NaN  NaN  NaN  NaN
14  NaN  NaN  NaN  NaN  NaN  NaN  NaN
15  NaN  NaN  NaN  NaN  NaN  NaN  NaN
16  NaN  NaN  NaN  NaN  NaN  NaN  NaN
17  NaN  NaN  NaN  NaN  NaN  NaN  NaN
18  NaN  NaN  NaN  NaN  NaN  NaN  NaN
19  NaN  NaN  NaN  NaN  NaN  NaN  NaN
20  NaN  NaN  NaN  NaN  NaN  NaN  NaN
21  NaN  NaN  NaN  NaN  NaN  NaN  NaN
22  NaN  NaN  NaN  NaN  NaN  NaN  NaN
23  NaN  NaN  NaN  NaN  NaN  NaN  NaN

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

hour = 1
area = 4

#determine the number of associates required by volume for the hour
numassociates[area,hour] = int(ceil(sum(Vol2[area,door,hour] for door in areaDoors[area] if Vol2[area,door,hour] > 0)/float(goal[area])))

#set the range of associates
associates = range(1,numassociates[area,hour]+1)

''' ----- DECISION VARIABLES -----  '''

#Assignment of doors to associates
#X = m.addVars(areaDoors[area],associates,vtype=GRB.BINARY,name='X')
X = {}
Z = {}
alpha = {}

for j in associates:
    Z[j] = cvx.Variable(boolean=True)
    alpha[j] = cvx.Variable()
    for i in areaDoors[area]:
        X[i,j] = cvx.Variable(boolean=True)

''' ----- OBJECTIVE FUNCTION ----- '''

obj = cvx.Minimize(sum(alpha[j] for j in associates))

''' ----- CONSTRAINTS ----- '''

'''For any associate in the set of associates that are calculated as needed in that area and hour, 
the sum of volume going to the doors that associate is assigned must be greater than or equal to 
the goal, minus the underachievement.'''
devFromGoal = [sum(Vol2[area,i, hour] * X[i,j] for i in areaDoors[area] if Vol2[area, i, hour] > 0) 
    >=  Z[j]*goal[area] - alpha[j] for j in associates]

'''Ensure that associates are only assigned to doors that are adjacent to each other. 
If a door has a volume of zero, it will skip the consideration of assigning an associate 
to that door, and thus the next adjacent door can be assigned.'''
doorAdjacency = [X[i2,j] >= X[i1,j] + X[i3,j] - 1 
    for j in associates for i1 in areaDoors[area] if Vol2[area,i1,hour]>0 
    for i2 in areaDoors[area] if Vol2[area,i2,hour]>0 for i3 in areaDoors[area] if Vol2[area,i3,hour]>0 
    if i1<i2<i3]

'''Ensure that a door is assigned to one associate and only one.'''
assignOnly1 = [sum(X[i,j] for j in associates) == 1 for i in areaDoors[area] if Vol2[area, i, hour] > 0]

'''Ensure that if a door has no volume in a given hour, it must not be assigned.'''
doNotAssignZeroVolDoor = [sum(X[i,j] for j in associates) == 0 for i in areaDoors[area] if Vol2[area, i, hour] == 0]

'''Binary switching constraints. If a door is assigned an associate, that associate must be considered as used.'''
usedBinarySwitch = [X[i,j] <= Z[j] for j in associates for i in areaDoors[area] if Vol2[area, i, hour] > 0]

'''If an associate is to be used, they must be assigned to at least one door.'''
assignedBinarySwitch = [sum(X[i,j] for i in areaDoors[area]) >= Z[j] for j in associates]

'''Ensure that the deviation for an associate does not exceed the goal for that associate (don't completely understand this) '''
IDK = [alpha[j] <= goal[area]*Z[j] for j in associates]

'''The model has to use the number of associates that are calculated as needed in the area and hour,
minus one associate if it can find a way to feasibly do so.'''
minusOneIfPossible = [sum(Z[j] for j in associates) >= numassociates[area,hour]-1]       

'''Ensure that the volume assigned to an associate across doors to not exceed a certain number of cartons.'''
loadLimit = [sum(Vol2[area,i,hour] * X[i,j] for i in areaDoors[area]) <= loadLimit for j in associates]        

'''Ensure that the number of doors an associate is assigned never exceeds a certain number.'''
lineLimit = [sum(X[i,j] for i in areaDoors[area]) <= lineLimit for j in associates]

#XLB = [X[i,j]>=0 for i in areaDoors[area] for j in associates]
#XUB = [X[i,j]<=1 for i in areaDoors[area] for j in associates]

ZLB = [Z[j]>=0 for j in associates]
alphaLB = [alpha[j]>=0 for j in associates]

constraints = devFromGoal+doorAdjacency+assignOnly1+doNotAssignZeroVolDoor+usedBinarySwitch+assignedBinarySwitch+IDK+minusOneIfPossible+loadLimit+lineLimit+ZLB+alphaLB


''' ----- BEGIN RUN ----- '''
print('Hour: ',hour)
print('Area: ',area)

#allow optimization to run in the background without displaying all the output; solve
prob = cvx.Problem(obj,constraints)
prob.solve(solver='ECOS_BB')

''' ----- POST-SOLUTION PROCESSING ----- '''

#calculate the load per used associate and whether associate was used
load = {}
used = {}
totalUsed = 0
for j in associates: 
    load[area,hour,j] = sum(Vol2[area,i,hour]*X[i,j].value for i in areaDoors[area] if Vol2[area,i,hour]>0 if X[i,j].value!=None)
    if load[area,hour,j] > 0:
        used[area,hour,j] = 1
    else: 
        used[area,hour,j] = 0
    totalUsed += used[area,hour,j]

#create a dataFrame for the hour, which will contain the associate to door assignments
assignments[area,hour] = pd.DataFrame(data=None,index=areaDoors[area],columns=associates)    
for i in areaDoors[area]:
     for j in associates:
        if X[i,j].value > 0.2:
            assignments[area,hour].loc[i,j] = X[i,j].value
        else:
            assignments[area,hour].loc[i,j] = ''
        '''for assignment in assignments:
            fileName = "".join(["resultsArea",str(area),"Hour",str(hour),".csv"])
            assignments[assignment].to_csv(fileName)'''
#calculate the area productivity
productivity = {}

''' ----- DISPLAY OUTPUT ----- '''

print('# Associates Required by Volume: ',numassociates[area,hour])
print('# Associates Used: ',totalUsed)
print('Assignments:')
print(assignments[area,hour])
print('Load per Associate: ')
for j in associates:
    #print j,alpha[j].value
    print(j,'(',used[area,hour,j],')',': ',load[area,hour,j],'|',alpha[j].value)
print('Total volume not expected to load (objective): ',prob.value)
print('++++++++++++++++++++')
print('')

In [ ]:
for i in areaDoors[7]:
    for j in associates:
        print X[i,j].value